In [1]:
%display latex

# Equation

$$ K^{2D}_{\alpha \beta} = \delta_{jk}\delta_{\alpha \beta}+ (1 - \delta_{jk})[ \delta_{\alpha \beta}H_0(k_0 r_{jk}) + (1-\delta_{\alpha \beta})e^{2\alpha i \phi_{jk}} H_2(k_0r_{jk})  ] $$

$\mathbf{r}_{jk} = \mathbf{r}_j - \mathbf{r}_k$  

$r_{jk} = |\mathbf{r}_{jk}|$  

$ \hat{r}_{jk} = \frac{\mathbf{r}_{jk}}{r_{jk}} = (cos\phi_{jk}, sin \phi_{jk}) $

$\phi_{jk} = atan(\frac{y}{x})$

# Analytical

In [2]:
N = 2
r_jk = matrix(SR, 2, var('r_11,r_12,r_21,r_22'))#
r_jk

[r_11 r_12]
[r_21 r_22]

In [3]:
H0 = matrix(SR, N)
H2 = matrix(SR, N)
for j in range(N):
    for k in range(N):
        H0[j,k] = hankel1(0, r_jk[j,k]) 
        H2[j,k] = hankel1(2, r_jk[j,k]) 
display(H0)
print()
display(H2)

[hankel1(0, r_11) hankel1(0, r_12)]
[hankel1(0, r_21) hankel1(0, r_22)]

[hankel1(2, r_11) hankel1(2, r_12)]
[hankel1(2, r_21) hankel1(2, r_22)]

In [4]:
ϕ = matrix(SR, 2, var('phi_11,phi_12,phi_21,phi_22'))
ϕ

[phi_11 phi_12]
[phi_21 phi_22]

## Creating each element following above definition

In [5]:
α_range = β_range = [-1,+1]
for α in α_range:
    for β in β_range:
        for j in range(N):
            for k in range(N):
                term1 = kronecker_delta(j,k)*kronecker_delta(α,β)
                term2 = 1 - kronecker_delta(j,k)
                term3 = kronecker_delta(α,β)*H0[j,k]
                term4 = 1-kronecker_delta(α,β)
                term5 = exp(2*α*I*ϕ[j,k])*H2[j,k]
                K = term1 + term2*( term3 + term4*term5  )
                print('----> α:', α, ' ... β:', β, ' ... j:', j, ' ... k:', k, ' ... ')
                display(K)
        print( )

----> α: -1  ... β: -1  ... j: 0  ... k: 0  ... 


1

----> α: -1  ... β: -1  ... j: 0  ... k: 1  ... 


hankel1(0, r_12)

----> α: -1  ... β: -1  ... j: 1  ... k: 0  ... 


...

hankel1(0, r_12)

----> α: 1  ... β: 1  ... j: 1  ... k: 0  ... 


hankel1(0, r_21)

----> α: 1  ... β: 1  ... j: 1  ... k: 1  ... 


1

## Creating smaller matrices

Compute all terms related at once, that is, each $\delta_{jk}$, which corresponds to a $N\times N$ matrix.

In [6]:
for (idx_α,α) in enumerate(α_range):
    for (idx_β,β) in enumerate(β_range):
        term1 = identity_matrix(N)*kronecker_delta(α,β)
        term2 = ones_matrix(N) - identity_matrix(N) # == (1 - delta_{jk})
        term3 = kronecker_delta(α,β)*H0
        term4 = 1 - kronecker_delta(α,β)
        term5 = ϕ.apply_map(exp(2*α*I*x)).elementwise_product(H2)
        K = term1 + term2.elementwise_product( term3  + term4*term5  )
        print( '----> α:', α, 'β:',β  )
        display(K)
        print()

----> α: -1 β: -1


[               1 hankel1(0, r_12)]
[hankel1(0, r_21)                1]


----> α: -1 β: 1


[                               0 e^(-2*I*phi_12)*hankel1(2, r_12)]
[e^(-2*I*phi_21)*hankel1(2, r_21)                                0]


----> α: 1 β: -1


[                              0 e^(2*I*phi_12)*hankel1(2, r_12)]
[e^(2*I*phi_21)*hankel1(2, r_21)                               0]


----> α: 1 β: 1


[               1 hankel1(0, r_12)]
[hankel1(0, r_21)                1]

# Numerical

In [7]:
N = 2
r = matrix([ [1,2],[3,5]  ] )
r

[1 2]
[3 5]

In [8]:
r_jk = matrix(SR, 2)
ϕ = matrix(SR, 2)

H0 = matrix(SR, N)
H2 = matrix(SR, N)

for j in range(N):
    for k in range(N):
        vec_r_jk = r[j] - r[k]
        r_jk[j,k] = norm(vec_r_jk)
        ϕ[j,k] = atan2( vec_r_jk[1], vec_r_jk[0] )
        
        H0[j,k] = hankel1(0, r_jk[j,k]) 
        H2[j,k] = hankel1(2, r_jk[j,k])
        
display(H0)
print()
display(H2)

[       hankel1(0, 0) hankel1(0, sqrt(13))]
[hankel1(0, sqrt(13))        hankel1(0, 0)]

[       hankel1(2, 0) hankel1(2, sqrt(13))]
[hankel1(2, sqrt(13))        hankel1(2, 0)]

In [9]:
for (idx_α,α) in enumerate(α_range):
    for (idx_β,β) in enumerate(β_range):
        term1 = identity_matrix(2)*kronecker_delta(α,β)
        term2 = ones_matrix(2) - identity_matrix(2)
        term3 = kronecker_delta(α,β)*H0
        term4 = 1 - kronecker_delta(α,β)
        term5 = ϕ.apply_map(exp(2*α*I*x)).elementwise_product(H2)
        K = term1 + term2.elementwise_product( term3  + term4*term5  )
        print( '----> α:', α, 'β:',β  )
        display( K )
        print()
         

----> α: -1 β: -1


[                   1 hankel1(0, sqrt(13))]
[hankel1(0, sqrt(13))                    1]


----> α: -1 β: 1


[                                        0 e^(-2*I*arctan(3/2))*hankel1(2, sqrt(13))]
[e^(-2*I*arctan(3/2))*hankel1(2, sqrt(13))                                         0]


----> α: 1 β: -1


[                                       0 e^(2*I*arctan(3/2))*hankel1(2, sqrt(13))]
[e^(2*I*arctan(3/2))*hankel1(2, sqrt(13))                                        0]


----> α: 1 β: 1


[                   1 hankel1(0, sqrt(13))]
[hankel1(0, sqrt(13))                    1]